## Import Libraries

In [1]:
# %matplotlib ipympl
# %matplotlib inline
%matplotlib wx

In [2]:
import matplotlib.pyplot as plt
plt.ion()

In [3]:
from pydgilib_extra import *
from atprogram.atprogram import atprogram

In [4]:
from os import getcwd, path, pardir
import pickle

## Compile and program project

In [5]:
project_path = path.curdir + "/" + "No-Security"
project_name =  "No-Security"
device_name = "ATSAML21J18B"
project_path

'./No-Security'

In [6]:
atprogram(project_path, device_name = device_name, verbose=3)

Building file: ../Device_Startup/system_saml21.c
Invoking: ARM/GNU C Compiler : 6.3.1
"C:\Program Files (x86)\Atmel\Studio\7.0\toolchain\arm\arm-gnu-toolchain\bin\arm-none-eabi-gcc.exe"  -x c -mthumb -D__SAML21J18B__ -DDEBUG  -I"C:\Program Files (x86)\Atmel\Studio\7.0\Packs\arm\CMSIS\5.4.0\CMSIS\Core\Include" -I"../Config" -I".." -I"../examples" -I"../hal/include" -I"../hal/utils/include" -I"../hpl/core" -I"../hpl/dmac" -I"../hpl/gclk" -I"../hpl/mclk" -I"../hpl/nvmctrl" -I"../hpl/osc32kctrl" -I"../hpl/oscctrl" -I"../hpl/pm" -I"../hpl/port" -I"../hpl/systick" -I"../hri" -I"C:\Program Files (x86)\Atmel\Studio\7.0\Packs\atmel\SAML21_DFP\1.2.125\saml21b\include"  -O1 -ffunction-sections -mlong-calls -g3 -Wall -mcpu=cortex-m0plus -c -std=gnu99 -MD -MP -MF "Device_Startup/system_saml21.d" -MT"Device_Startup/system_saml21.d" -MT"Device_Startup/system_saml21.o"   -o "Device_Startup/system_saml21.o" "../Device_Startup/system_saml21.c" 
Finished building: ../Device_Startup/system_saml21.c
Buildi

0

## Data Logging

In [7]:
live_plot = False

Create a figure for the plot.

In [8]:
if live_plot:
    fig = plt.figure(figsize=(10, 6))
    fig.show()

Create the configuration dictionary for `DGILibExtra`.

In [9]:
config_dict = {
    "loggers": [LOGGER_OBJECT, LOGGER_CSV],
    "file_name_base": "experiment_no_security_flash"
}
config_dict_plot = {
    "loggers": [LOGGER_OBJECT, LOGGER_PLOT, LOGGER_CSV],
    "plot_pins": [False, False, True, True],
    "plot_pins_method": "line",
    "plot_xmax": 5,
    "window_title": "Experiment No Security",
}

Stop criteria to pass to the logger:

In [10]:
def stop_fn(logger_data):
    return all(logger_data.gpio.values[-1])

Perform the measurement.

In [11]:
data = []
cd = config_dict.copy()
if live_plot:
    fig.clf()
    for ax in fig.get_axes():
        ax.cla()
    
    cd.update(config_dict_plot)
    cd["fig"] = fig
    
with DGILibExtra(**cd) as dgilib:
    dgilib.device_reset()
    dgilib.logger.log(1000,stop_fn)
    data = dgilib.data

# Store Data

In [12]:
import pickle

In [13]:
pickle.dump(data, open("no_security_flash_logger_data.p", "wb"))

# Load Data

In [14]:
data = pickle.load(open("no_security_flash_logger_data.p", "rb"))

In [15]:
#iteration = 0
#name = "No_Security_Flash"
#data = pickle.load(open(path.join(path.pardir, path.pardir, f"{name}_{iteration}.p"), "rb"))

## Analysis

Create Stop Function to stop parsing the data when all pins are high.

In [16]:
def stop_function(pin_values):
    return all(pin_values)

Parse the data.

In [17]:
flash_write_charge, flash_write_time = power_and_time_per_pulse(data, 2, stop_function=stop_function)

In [18]:
flash_read_charge, flash_read_time = power_and_time_per_pulse(data, 3, stop_function=stop_function)

In [19]:
print(len(flash_write_charge), len(flash_write_time), len(flash_read_charge), len(flash_read_time))

6000 6000 6000 6000


In [20]:
drop = 1
cutoff = max(len(flash_write_charge), len(flash_write_time), len(flash_read_charge), len(flash_read_time)) - drop
flash_write_charge = flash_write_charge[:cutoff]
flash_write_time = flash_write_time[:cutoff]
flash_read_charge = flash_read_charge[:cutoff]
flash_read_time = flash_read_time[:cutoff]

# Convert to Joule

In [21]:
voltage = 3.31
j_scale = 1e3 # m
t_scale = 1e3 # m
model_j_scale = 1e6 # n
model_t_scale = 1e3 # u

experiment_name = 'No Security'

In [22]:
flash_write_energy = flash_write_charge[:cutoff]
flash_read_energy = flash_read_charge[:cutoff]
flash_write_time_s = flash_write_time[:cutoff]
flash_read_time_s = flash_read_time[:cutoff]

for i in range(len(flash_write_energy)):
    flash_write_energy[i] = flash_write_energy[i] * voltage * j_scale
for i in range(len(flash_read_energy)):
    flash_read_energy[i] = flash_read_energy[i] * voltage * j_scale
    
for i in range(len(flash_write_time_s)):
    flash_write_time_s[i] = flash_write_time_s[i] * t_scale
for i in range(len(flash_read_time_s)):
    flash_read_time_s[i] = flash_read_time_s[i] * t_scale

In [23]:
MIN_NUM_BYTES = 1
num_bytes = range(MIN_NUM_BYTES, MIN_NUM_BYTES + len(flash_write_charge))
print(f"MAX_NUM_BYTES: {num_bytes[-1]}")

MAX_NUM_BYTES: 5999


In [24]:
from lmfit import Model

def line(x, slope, intercept):
    """a line"""
    return [slope*i + intercept for i in x]

mod = Model(line)
pars = mod.make_params(slope=0, intercept=1)
# pars['intercept'].set(min=0)

In [25]:
results = []
ylabels = ['Energy [mJ]'] * 2 + ['Time [ms]'] * 2 + ['Energy [mJ]'] + ['Time [ms]']
parameter_names = [
    'Flash Write Energy',
    'Flash Read Energy',
    'Flash Write Time',
    'Flash Read Time',
    'Total Energy',
    'Total Time',
]
for y in [flash_write_energy, flash_read_energy, flash_write_time_s, flash_read_time_s, 
          [w + r for (w,r) in zip(flash_write_energy, flash_read_energy)],
          [w + r for (w,r) in zip(flash_write_time_s, flash_read_time_s)]]:
#     result = mod.fit(y[16:-16], pars, x=num_bytes[16:-16])
    result = mod.fit(y, pars, x=num_bytes) 
    print(result.fit_report())
    fig, grid = result.plot(
        xlabel='Checkpoint Size [Bytes]',
        ylabel=ylabels[len(results)])
    fig.tight_layout(rect=(0.05, 0.05, 1, 1))
    fig.set_size_inches(5, 4.5, forward=True)
    fig.canvas.set_window_title(
        f"Residuals of {experiment_name} {parameter_names[len(results)]}")
    fig.show()
    results.append(result)

[[Model]]
    Model(line)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 6
    # data points      = 5999
    # variables        = 2
    chi-square         = 0.38626876
    reduced chi-square = 6.4410e-05
    Akaike info crit   = -57889.7692
    Bayesian info crit = -57876.3705
[[Variables]]
    slope:      1.3167e-04 +/- 5.9834e-08 (0.05%) (init = 0)
    intercept:  0.01922280 +/- 2.0726e-04 (1.08%) (init = 1)
[[Correlations]] (unreported correlations are < 0.100)
    C(slope, intercept) = -0.866
[[Model]]
    Model(line)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 6
    # data points      = 5999
    # variables        = 2
    chi-square         = 6.4585e-05
    reduced chi-square = 1.0770e-08
    Akaike info crit   = -110058.879
    Bayesian info crit = -110045.480
[[Variables]]
    slope:      2.9225e-06 +/- 7.7370e-10 (0.03%) (init = 0)
    intercept: -3.1292e-05 +/- 2.6801e-06 (8.56%) (init = 1)
[[Correlations]] (unrepo

In [26]:
fig2 = plt.figure(figsize=(8, 6))
fig2.canvas.set_window_title(f"Analysis {experiment_name}")

In [27]:
charge_color = 'r'
time_color = 'b'

In [28]:
fig2.clf()
# fig2.suptitle("Energy analysis of AES")
ax1 = fig2.add_subplot(1, 1, 1)
ax2 = ax1.twinx()
ax1.set_xlabel('Checkpoint Size [Bytes]')
ax1.set_ylabel('Energy [mJ]', color=charge_color)
ax2.set_ylabel('Time [ms]', color=time_color)
ax1.tick_params('y', colors=charge_color)
ax2.tick_params('y', colors=time_color)

In [29]:
lines = []
lines += ax1.plot(num_bytes, flash_write_energy, charge_color+'-', label=f'{parameter_names[len(lines)]}')
lines += ax1.plot(num_bytes, flash_read_energy, charge_color+'--', label=f'{parameter_names[len(lines)]}')
lines += ax2.plot(num_bytes, flash_write_time_s, time_color+'-', label=f'{parameter_names[len(lines)]}')
lines += ax2.plot(num_bytes, flash_read_time_s, time_color+'--', label=f'{parameter_names[len(lines)]}')
ax1.legend(handles=lines)
# [flash_write_charge, flash_read_charge, flash_write_time, flash_read_time]:
ax1.set_title(
    f"{parameter_names[0]}: Slope {results[0].params['slope'].value * model_j_scale:.04} nJ/B, Intercept {results[0].params['intercept'].value * model_j_scale:.04} nJ\n" +
    f"{parameter_names[1]}: Slope {results[1].params['slope'].value * model_j_scale:.04} nJ/B, Intercept {results[1].params['intercept'].value * model_j_scale:.04} nJ\n" +
    f"{parameter_names[2]}: Slope {results[2].params['slope'].value * model_t_scale:.04} $\mu$s/B, Intercept {results[2].params['intercept'].value * model_t_scale:.04} $\mu$s\n" +
    f"{parameter_names[3]}: Slope {results[3].params['slope'].value * model_t_scale:.04} $\mu$s/B, Intercept {results[3].params['intercept'].value * model_t_scale:.04} $\mu$s\n" +
    f"{parameter_names[4]}: Slope {results[4].params['slope'].value * model_j_scale:.04} nJ/B, Intercept {results[4].params['intercept'].value * model_j_scale:.04} nJ\n" +
    f"{parameter_names[5]}: Slope {results[5].params['slope'].value * model_t_scale:.04} $\mu$s/B, Intercept {results[5].params['intercept'].value * model_t_scale:.04} $\mu$s\n")
fig2.tight_layout()
fig2.show()

In [30]:
print(
    f"{parameter_names[0]}: Slope {results[0].params['slope'].value * model_j_scale:.020} nJ/B, Intercept {results[0].params['intercept'].value * model_j_scale:.020} nJ\n" +
    f"{parameter_names[1]}: Slope {results[1].params['slope'].value * model_j_scale:.020} nJ/B, Intercept {results[1].params['intercept'].value * model_j_scale:.020} nJ\n" +
    f"{parameter_names[2]}: Slope {results[2].params['slope'].value * model_t_scale:.020} $\mu$s/B, Intercept {results[2].params['intercept'].value * model_t_scale:.020} $\mu$s\n" +
    f"{parameter_names[3]}: Slope {results[3].params['slope'].value * model_t_scale:.020} $\mu$s/B, Intercept {results[3].params['intercept'].value * model_t_scale:.020} $\mu$s\n" +
    f"{parameter_names[4]}: Slope {results[4].params['slope'].value * model_j_scale:.020} nJ/B, Intercept {results[4].params['intercept'].value * model_j_scale:.020} nJ\n" +
    f"{parameter_names[5]}: Slope {results[5].params['slope'].value * model_t_scale:.020} $\mu$s/B, Intercept {results[5].params['intercept'].value * model_t_scale:.020} $\mu$s\n"
)

Flash Write Energy: Slope 131.67037688734066592 nJ/B, Intercept 19222.804547851079406 nJ
Flash Read Energy: Slope 2.9225410016964255 nJ/B, Intercept -31.291770743235858276 nJ
Flash Write Time: Slope 14.006059232132310299 $\mu$s/B, Intercept 2036.2728066915610725 $\mu$s
Flash Read Time: Slope 0.70670176512217119935 $\mu$s/B, Intercept 7.8310803415690966034 $\mu$s
Total Energy: Slope 134.59291788903470888 nJ/B, Intercept 19191.512779298314854 nJ
Total Time: Slope 14.712760997254845208 $\mu$s/B, Intercept 2044.1038870355473591 $\mu$s



In [31]:
# Save Charge amount list into pickle file
import pickle
pickle.dump(flash_write_energy, open("no_security_flash_write_energy_mJ.p", "wb"))
pickle.dump(flash_read_energy, open("no_security_flash_read_energy_mJ.p", "wb"))
pickle.dump(flash_write_time_s, open("no_security_flash_write_time_ms.p", "wb"))
pickle.dump(flash_read_time_s, open("no_security_flash_read_time_ms.p", "wb"))

## Write config file

In [32]:
import json

config = {}
config["name"] = "No Security Flash"
config["project_paths"] = [project_path]
config["config_dict"] = config_dict
config["config_dict_plot"] = config_dict_plot
config["analysis"] = {"pins":{2: ["No Security Flash Write"], 3: ["No Security Flash Read"]}, 
                      "result_types": ["Charge", "Time"],
                      "section_types": {"init": [], 
                                        "store": ["No Security Flash Write"],
                                        "load": ["No Security Flash Read"],
                                        "exit": []},
                      "labels": {
                          "Charge": {"x":"Data Size", "x_unit": "byte", "y": "Charge", "y_unit": "C"},
                          "Time": {"x":"Data Size", "x_unit": "byte", "y": "Time", "y_unit": "s"},
                      },
                      "x_step": 1}

with open("looped_experiment.json", 'w') as config_file:  
    json.dump(config, config_file, indent=4)

# Write model data

In [33]:
dump_pickle = True
fit_lm = True
verbose = 2
show_lm_plot = 2

In [34]:
# Parse data
analysis_config = config.get("analysis")
result_types = analysis_config.get("result_types")
x_step = analysis_config.get("x_step")
parsed_data = {}
for pin, parameter_names in analysis_config.get("pins").items():
    data2 = power_and_time_per_pulse(
        data, int(pin), stop_function=stop_function)
    num_names = len(parameter_names)
    for i, parameter_name in enumerate(parameter_names):
        end_index = -drop * num_names or None
        parsed_data[parameter_name] = {
            result_types[0]: data2[0][i:end_index:num_names],
            result_types[1]: data2[1][i:end_index:num_names],
            "x_step": x_step}
if dump_pickle:
    pickle.dump(parsed_data, open(
        path.join(path.curdir,
                  f"{config_dict.get('file_name_base')}_looped.p"), "wb"))

In [35]:
# Fit lm
if fit_lm:
    model = None
    if model is None:
        def line(x, intercept, slope):
            """a line"""
            return [intercept + slope*i for i in x]

        model = Model(line)
        params = model.make_params(intercept=0, slope=1)
#         params['intercept'].set(min=0)
    else:
        params = model.params

    model_results = {}
    labels = analysis_config.get("labels")
    for parameter_name in parsed_data.keys():
        length = len(parsed_data[parameter_name][result_types[0]])
        x_step = parsed_data[parameter_name]["x_step"]
        num_bytes = range(x_step, (length+1)*x_step, x_step)
        if verbose:
            print(
                f"Fitting model to {parameter_name} with {length} " +
                f"samples, from {min(num_bytes)} to {max(num_bytes)} "
                f"bytes in steps of {x_step}.")
        model_result = {}
        for result_type in result_types:
            model_result[result_type] = model.fit(
                parsed_data[parameter_name][result_type], params,
                x=num_bytes)
            if verbose >= 2:
                print(model_result[result_type].fit_report())
            # Plot multiple view
            if show_lm_plot >= 2:
                fig, grid = model_result[result_type].plot(
                    xlabel=f"{labels[result_type]['x']} " +
                           f"[{labels[result_type]['x_unit']}]",
                    ylabel=f"{labels[result_type]['y']} " +
                           f"[{labels[result_type]['y_unit']}]")
                fig.canvas.set_window_title(
                    f"Residuals of {parameter_name}")
                fig.tight_layout()
                fig.show()
            model_results[parameter_name] = model_result

    # Plot single view
    if show_lm_plot:
        import matplotlib.pyplot as plt
        fig = plt.figure(figsize=(9, 6))
        fig.canvas.set_window_title(f"Analysis {config.get('name')}")
        colors = dict(zip(result_types, ['r', 'b']))
        line_styles = (
            line_style for line_style in ('-', '--', '-.', ':') * 2)
        # fig.suptitle(f"Energy analysis of {config.get('name')}")
        ax = {}
        ax[result_types[0]] = fig.add_subplot(1, 1, 1)
        ax[result_types[1]] = ax[result_types[0]].twinx()
        ax[result_types[0]].set_xlabel(
            f"{labels[result_types[0]]['x']} " +
            f"[{labels[result_types[0]]['x_unit']}]")
        for result_type in result_types:
            ax[result_type].set_ylabel(
                f"{labels[result_type]['y']} " +
                f"[{labels[result_type]['y_unit']}]",
                color=colors[result_type])
            ax[result_type].tick_params('y', colors=colors[result_type])

        lines = []
        title_str = ""
        for parameter_name in parsed_data.keys():
            length = len(parsed_data[parameter_name][result_types[0]])
            x_step = parsed_data[parameter_name]["x_step"]
            num_bytes = range(x_step, (length+1)*x_step, x_step)
            model_result = {}
            line_style = next(line_styles)
            for result_type in result_types:
                label = f"{parameter_name} {labels[result_type]['y']}"
                lines += ax[result_type].plot(
                    num_bytes, parsed_data[parameter_name][result_type],
                    colors[result_type] + line_style, label=label)
                title_str += f"{label} "
                for param in params.keys():
                    title_str += "".join(
                        f"{params[param].name.capitalize()}: ")
                    title_str += "".join(
                        f"{model_results[parameter_name][result_type].params[param].value: .03} ")
                    title_str += "".join(
                        f"{labels[result_type]['y_unit']}, ")
                title_str = title_str[:-2] + \
                    f" per {labels[result_type]['x_unit']}\n"
        ax[result_types[0]].legend(handles=lines)
        ax[result_types[0]].set_title(title_str[:-1])
        # fig.tight_layout()
        fig.tight_layout(rect=(0.05, 0.05, 1, 1))
        fig.set_size_inches(8, 6, forward=True)
        fig.show()

    # Save model results to file
    if dump_pickle:
        model_results_dump = {}
        for parameter_name in model_results.keys():
            model_results_dump[parameter_name] = {}
            for result_type in model_results[parameter_name].keys():
                model_results_dump[parameter_name][result_type] = \
                    model_results[parameter_name][result_type].values

        pickle.dump(model_results_dump, open(path.join(
            path.curdir,
            f"{config_dict.get('file_name_base')}_model.p"), "wb"))

Fitting model to No Security Flash Write with 5999 samples, from 1 to 5999 bytes in steps of 1.
[[Model]]
    Model(line)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 9
    # data points      = 5999
    # variables        = 2
    chi-square         = 3.5256e-08
    reduced chi-square = 5.8789e-12
    Akaike info crit   = -155130.001
    Bayesian info crit = -155116.603
[[Variables]]
    intercept:  5.8075e-06 +/- 6.2617e-08 (1.08%) (init = 0)
    slope:      3.9780e-08 +/- 1.8077e-11 (0.05%) (init = 1)
[[Correlations]] (unreported correlations are < 0.100)
    C(intercept, slope) = -0.866
[[Model]]
    Model(line)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 7
    # data points      = 5999
    # variables        = 2
    chi-square         = 0.00425637
    reduced chi-square = 7.0975e-07
    Akaike info crit   = -84933.9586
    Bayesian info crit = -84920.5599
[[Variables]]
    intercept:  0.00203627 +/- 2.1757e-05 (1.07%) 

# Total from measurement

In [36]:
n_samples = 5995

def avg(l):
    return sum(l)/len(l)

avg_energy = sum(flash_write_energy[:n_samples]) + sum(flash_read_energy[:n_samples])
avg_time = sum(flash_write_time_s[:n_samples]) + sum(flash_read_time_s[:n_samples])
print(total_energy, total_time)

NameError: name 'total_energy' is not defined